In [20]:
import pandas as pd
from datetime import datetime
from dateutil.parser import parse
from django.core.exceptions import ObjectDoesNotExist, FieldError
file = "archiv\data\Database_Vorarlberg_Correction_Progress.xlsx"
df = pd.read_excel(file, "RESEARCH")[2:].fillna('False')

In [21]:
df

,*SiteID,*Research Activity ID,Start Date,End Date,Responsible Researcher,Responsible Institution,When was the data-set generated?,Activity-ID,Research Question,Comment,Research type,Research Method,EntityID,Public,EnteredBy
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,OE_06001,OE_06001.R0001,01012014,31122014,"Rudi Hinterwaldner, Claus-Stephan Holdermann",CONTEXT OG,False,92109.15.01,Usage of the area,False,scientific research,"metal detecting survey, core drilling, test pit",OE_06001.E0001,yes,Patricia
4,OE_06001,OE_06001.R0002,01011826,31121826,False,False,False,False,False,False,False,excavation,OE_06001.E0002,False,False
5,OE_06001,OE_06001.R0003,01011861,31121861,Douglaß,False,False,False,False,False,False,excavation,OE_06001.E0002,False,False
6,OE_06001,OE_06001.R0004,01011939,31121941,Adolf Hild,False,False,False,False,False,False,excavation,OE_06001.E0002,False,False
7,OE_06001,OE_06001.R0005,01011947,31121947,Adolf Hild,False,False,False,False,False,False,excavation,OE_06001.E0002,False,False
8,OE_06001,OE_06001.R0006,01101934,31101934,Adolf Hild,False,False,False,chance find,False,False,field-walking survey,OE_06001.E0002,False,False
9,OE_06001,OE_06001.R0007,27081937,28081937,Adolf Hild,False,False,False,Search for possible settlement areas.,False,False,excavation,OE_06001.E0003,False,False
10,OE_06002,OE_06002.R0001,01092014,30112014,Maria Bader,TALPA GnbR,False,92117.14.01,False,False,scientific research,"stratigraphic excavation, artefact studies",OE_06002.E0001,yes,Patricia
11,OE_06002,OE_06002.R0002,01062013,30062013,Andreas Picker,BDA Vorarlberg,False,-,chance find during construction work,The bronze Certosafibula (Typ 10 after Terzan)...,development led research,"construction work - stray find, artefact studies",OE_06002.E0002,False,False


In [40]:
failed_sites = []
for i, row in df.iterrows():
    temp_item = None
    try:
        temp_site = Site.objects.get(identifier=row['*SiteID'])
    except ObjectDoesNotExist:
        failed_sites.append(row['*SiteID'])
        temp_site = None
    if temp_site:
        if row['*Research Activity ID'] != 'False':
            temp_item, _ = ResearchEvent.objects.get_or_create(
                legacy_research_id=row['*Research Activity ID']
            )
            temp_item.site_id.add(temp_site)
            

        if row['Start Date'] != 'False':
            try:
                start_date = datetime.strptime(str(row['Start Date']), '%d%m%Y')
            except ValueError:
                start_date = None
            if start_date:
                temp_item.start_date = start_date
        
        if row['End Date'] != 'False':
            try:
                end_date = datetime.strptime(str(row['End Date']), '%d%m%Y')
            except ValueError:
                end_date = None
            if end_date:
                temp_item.end_date = end_date
        
        
        if row['Responsible Researcher'] != 'False':
            x = None
            temp_person = None
            for x in row['Responsible Researcher'].split(','):
                x = x.strip()
                temp_person, _ = Person.objects.get_or_create(written_name=x)
                if len(x.split(' ')) == 2:
                    temp_person.forename = x.split(' ')[0]
                    temp_person.name = x.split(' ')[1]
                elif len(x.split(' ')) == 1:
                    temp_person.name = x
                temp_person.save()
                temp_item.responsible_researcher.add(temp_person)
        
        if row['Responsible Institution'] != 'False':
            x = None
            temp_inst = None
            for x in row['Responsible Institution'].split(','):
                x = x.strip()
                temp_inst, _ = Institution.objects.get_or_create(legacy_id=x)
                temp_inst.written_name = x
                temp_inst.save()
                temp_item.responsible_institution.add(temp_inst)
        
        if row['Activity-ID'] != 'False':
            temp_item.identifier = "{}".format(row['Activity-ID'])
            
        temp_item.save()

In [41]:
ResearchEvent.objects.all().delete()

(356,
 {'archiv.ResearchEvent': 78,
  'archiv.ResearchEvent_responsible_institution': 28,
  'archiv.ResearchEvent_responsible_researcher': 94,
  'archiv.ResearchEvent_site_id': 78,
  'idprovider.IdProvider': 78})